# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pul-e 'alam,33.9953,69.0227,294.56,27,0,2.02,AF,1718047082
1,1,tarawa,1.4190,172.9840,301.20,89,40,1.54,KI,1718047082
2,2,fort bragg,35.1390,-79.0060,303.61,41,0,3.60,US,1718047082
3,3,utrik,11.2278,169.8474,301.42,77,21,8.19,MH,1718047082
4,4,albany,42.6001,-73.9662,292.55,67,100,0.89,US,1718047082


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = "Wind Speed"
)
# Display the map
# YOUR CODE HERE
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Wind Speed,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] <= 300) & (city_data_df['Max Temp'] >= 291) & (city_data_df['Wind Speed'] <= 5) & (city_data_df['Humidity'] <= 80)]
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather = ideal_weather.dropna()
# Display sample data
# YOUR CODE HERE
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pul-e 'alam,33.9953,69.0227,294.56,27,0,2.02,AF,1718047082
4,4,albany,42.6001,-73.9662,292.55,67,100,0.89,US,1718047082
9,9,zhoukou,33.6333,114.6333,298.91,52,100,2.85,CN,1718047083
19,19,bethel,41.3712,-73.4140,296.75,56,100,2.57,US,1718047083
42,42,kitui,-1.3670,38.0106,292.45,70,75,4.63,KE,1718047084
...,...,...,...,...,...,...,...,...,...,...
564,564,aksu,41.1231,80.2644,297.81,30,0,2.50,CN,1718047109
576,576,osorio,-29.8867,-50.2697,295.51,73,1,2.03,BR,1718046876
579,579,belyayevka,51.3974,56.4096,292.48,74,35,1.57,RU,1718047110
582,582,kigorobya,1.6162,31.3089,294.59,70,93,1.85,UG,1718047110


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = " "
# Display sample data
# YOUR CODE HERE
hotel_df

C:\Users\solan\AppData\Local\Temp\ipykernel_30768\2208680102.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
0,pul-e 'alam,AF,33.9953,69.0227,27,
4,albany,US,42.6001,-73.9662,67,
9,zhoukou,CN,33.6333,114.6333,52,
19,bethel,US,41.3712,-73.4140,56,
42,kitui,KE,-1.3670,38.0106,70,
...,...,...,...,...,...,...
564,aksu,CN,41.1231,80.2644,30,
576,osorio,BR,-29.8867,-50.2697,73,
579,belyayevka,RU,51.3974,56.4096,74,
582,kigorobya,UG,1.6162,31.3089,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "limit":limit,
    "type":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = hotel_df.loc[index, 'Lat']
    Longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pul-e 'alam - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
zhoukou - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
kitui - nearest hotel: Parkside Hotel
lompoc - nearest hotel: Red Roof Inn Lompoc
piacabucu - nearest hotel: O Leão
nova vicosa - nearest hotel: Pousada Pontal da Barra
badger - nearest hotel: Clarion Hotel & Suites
thompson - nearest hotel: Thompson Inn
watertown - nearest hotel: Four Points by Sheraton
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
shanhaiguan - nearest hotel: Bo Yi Business Hotel
pingliang - nearest hotel: 平凉新都酒店
mahina - nearest hotel: Motu Martin
aladag - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
callao - nearest hotel: Casa Ronald
minas de marcona - nearest hotel: No hotel found
novoazovs'k - nearest hotel: База відпочинку "Парус"
praia da vitoria - nearest hotel: Salles
alofi - nearest hotel: Matavai Resort
greenland -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pul-e 'alam,AF,33.9953,69.0227,27,No hotel found
4,albany,US,42.6001,-73.9662,67,No hotel found
9,zhoukou,CN,33.6333,114.6333,52,No hotel found
19,bethel,US,41.3712,-73.4140,56,Hampton Inn Danbury
42,kitui,KE,-1.3670,38.0106,70,Parkside Hotel
...,...,...,...,...,...,...
564,aksu,CN,41.1231,80.2644,30,Akesu Tianyuan International Hotel
576,osorio,BR,-29.8867,-50.2697,73,Big Hotel
579,belyayevka,RU,51.3974,56.4096,74,Уютный
582,kigorobya,UG,1.6162,31.3089,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_ideal = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = 'Humidity',
    hover_cols = ['Hotel Name', 'Country','City']
)
# Display the map
# YOUR CODE HERE
map_ideal

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country,City)